In [1]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

In [ ]:
DATASET_NAME = ""

In [3]:
results_embeddings = pd.read_csv('../../Embeddings/CNN_Image_Retrieval/{}-top-100-results-and-scores.csv'.format(DATASET_NAME))

In [5]:
def text_parser(text):
    new_text = ""
    for letter in text:
        if(not(letter in(['[',']',',']))):
            new_text+=letter
    new_text= np.array(new_text.split(' '))
    new_text= new_text.astype(float)
    return new_text

In [6]:
def parse_array(arr):
    new_arr = []
    for i in range(0,100):
        temp_arr = arr[i*2048:(i+1)*2048]
        new_arr.append(temp_arr)
    
    new_arr = np.array(new_arr)
    return new_arr

In [7]:
def get_closest(searched, embedding_list):
    smallest_distance = np.linalg.norm(searched-embedding_list[0,:])
    current_closest_idx = 0

    for embedding_idx in range(1, embedding_list.shape[0]):
        distance = np.linalg.norm(searched - embedding_list[embedding_idx,:])
        if(distance < smallest_distance):
            smallest_distance = distance
            current_closest_idx = embedding_idx
    
    return current_closest_idx

In [8]:
closest_queries = []
for query_idx in range(len(results_embeddings)):
    returned_embeddings = results_embeddings.iloc[query_idx]['result_emb']
    returned_embeddings = parse_array(text_parser(returned_embeddings))
    avg_embed = np.mean(returned_embeddings, axis = 0)
    closest_idx = get_closest(avg_embed, returned_embeddings)
    closest_queries.append(closest_idx)
    #print(query_idx+ 1)


In [9]:
def text_parse_paths(text):
    new_text = ""
    for letter in text:
        if(not(letter in(['[',']',',']))):
            new_text+=letter
    new_text= np.array(new_text.split(' '))
    return new_text

In [10]:
new_query_names  = []
for query_idx in range(len(results_embeddings)):
    image_name = text_parse_paths(results_embeddings.iloc[query_idx]['results_path'])[closest_queries[query_idx]]
    new_query_names.append(image_name)

new_query_names = [ Path(name).stem for name in new_query_names]

In [14]:
file = open('/../../Datasets/caltech101/gnd_caltech101_700.pkl','rb')
new_gt_file = open('/../../Datasets/gnd_caltech101_700_drift_cnn.pkl','wb')
gt_file = pickle.load(file)

In [15]:
new_gts = [{
'bbx' : None,
'easy' : [0],
'hard' : [],
'junk' : [],
'class' : '',
} for name in new_query_names]

In [16]:
new_gt_content = {
    'qimlist' : new_query_names,
    'imlist' : gt_file['imlist'],
    'gnd': new_gts}

In [17]:
pickle.dump(new_gt_content, new_gt_file, protocol = pickle.HIGHEST_PROTOCOL)

In [18]:
file.close()
new_gt_file.close()